In [ ]:
from google.colab import drive
# drive.mount('/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data')
drive.mount('/content/drive')

%cd drive/MyDrive/IITP/sohyun/creditcard_prediction/data
!pwd

Mounted at /content/drive
/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data
/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data


In [ ]:
# !pip install wandb -qqq
# import wandb
# wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import time
import datetime
from torch.autograd import grad
from torch.autograd import Variable
# from model import *
import matplotlib.pyplot as plt
# from utils import *
# from data_loader import *
import IPython
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler
import itertools
# from utils import *

## Dataset

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, eval_mode):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode :
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x), 0
        
    def __len__(self):
        return len(self.df)

# train_dataset = MyDataset(df=train_df, eval_mode=False)
# train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)

# val_dataset = MyDataset(df = val_df, eval_mode=True)
# val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False)


def get_loader(data_path, batch_size, mode='train'):
    """Build and return data loader."""
    train_df = pd.read_csv('./train.csv')
    train_df = train_df.drop(columns=['ID'])
    val_df = pd.read_csv('./val.csv')
    val_df = val_df.drop(columns=['ID'])
    test_df = pd.read_csv('./test.csv')
    test_df = test_df.drop(columns=['ID'])

    #-------------------#
    #---# Normalize #---#
    #-------------------#
    # case 1 - standardscaler
    scaler_n = StandardScaler()
    scaler_n.fit(train_df.values)
    
    val_x = val_df.drop(columns=['Class'])
    train_x_scaleN = pd.DataFrame(scaler_n.transform(train_df.values), columns = train_df.columns) # 확인 : train_x_scaleN.mean(), train_x_scaleN.var()
    val_x_scaleN = pd.DataFrame(scaler_n.transform(val_x.values), columns = val_x.columns)
    test_x_scaleN = pd.DataFrame(scaler_n.transform(test_df.values), columns = test_df.columns)

    train_df = train_x_scaleN
    val_df = pd.concat([val_x_scaleN, pd.DataFrame(val_df['Class'])])
    test_df = test_x_scaleN
    
    # dataset = MyDataset(data_path, mode)
    train_dataset = MyDataset(df=train_df, eval_mode=False)
    valid_dataset = MyDataset(df=val_df, eval_mode=True)
    test_dataset = MyDataset(df=test_df, eval_mode=False)
    
    shuffle = False
    if mode == 'train': shuffle = True

    data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=shuffle)
    data_loader_v = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=shuffle)
    data_loader_test = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=shuffle)
    return data_loader, data_loader_v, data_loader_test


In [ ]:
class EarlyStopping:
  def __init__(self, patience=30):
      self.loss = np.inf
      self.patience = 0
      self.patience_limit = patience
      
  def step(self, loss):
      if self.loss > loss:
          self.loss = loss
          self.patience = 0
      else:
          self.patience += 1
  
  def is_stop(self):
      return self.patience >= self.patience_limit

In [ ]:
class Solver(object):
  DEFAULTS = {}   
  def __init__(self, device, data_loader, data_loader_v, data_loader_test, config):
    # Data loader
    self.__dict__.update(Solver.DEFAULTS, **config)
    self.data_loader = data_loader
    self.data_loader_v = data_loader_v
    self.data_loader_test = data_loader_test
    self.device = device
    self.lr = 1e-4

    # Build tensorboard if use
    self.build_model()
    if self.use_tensorboard:self.build_tensorboard()

    # Start with trained model
    if self.pretrained_model: self.load_pretrained_model()

  def build_model(self):
    # Define model
    self.dagmm = DaGMM(self.device, self.gmm_k)

    # Optimizers
    self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr)

    # Print networks
    self.print_network(self.dagmm, 'DaGMM')

    if torch.cuda.is_available():
        self.dagmm.cuda()

  def print_network(self, model, name):
    num_params = 0
    for p in model.parameters():
        num_params += p.numel()
    print(name)
    # print(model)
    print("The number of parameters: {}".format(num_params))

  def load_pretrained_model(self):
    self.dagmm.load_state_dict(torch.load(os.path.join(
        self.model_save_path, '{}_dagmm.pth'.format(self.pretrained_model))))

    print("phi", self.dagmm.phi,"mu",self.dagmm.mu, "cov",self.dagmm.cov)
    print('loaded trained models (step: {})..!'.format(self.pretrained_model))

  # def build_tensorboard(self):
  #     from logger import Logger
  #     self.logger = Logger(self.log_path)

  def reset_grad(self):
    self.dagmm.zero_grad()

  def to_var(self, x, volatile=False):
    if torch.cuda.is_available() : x = x.cuda()
    return Variable(x, volatile=volatile)

  def train(self):
    iters_per_epoch = len(self.data_loader)

    # Start with trained model if exists
    if self.pretrained_model:
        start = int(self.pretrained_model.split('_')[0])
    else:
        start = 0

    # Start training
    iter_ctr = 0
    start_time = time.time()

    self.ap_global_train = np.array([0,0,0])
    best_f_score = -1
    early_stop = EarlyStopping(patience=30) # EarlyStopping
    
    for e in range(start, self.num_epochs):
      progress_bar = tqdm(self.data_loader)
      for i, (input_data, labels) in enumerate(progress_bar):
        iter_ctr += 1
        start = time.time()

        input_data = self.to_var(input_data) # cuda로
        total_loss, sample_energy, recon_error, cov_diag = self.dagmm_step(input_data)
        progress_bar.set_description(f'{e+1} epoch\'s total_loss : {total_loss.data.item()}')

        # Logging
        loss = {}
        loss['total_loss'] = total_loss.data.item()
        loss['sample_energy'] = sample_energy.item()
        loss['recon_error'] = recon_error.item()
        loss['cov_diag'] = cov_diag.item()

        # Print out log info
        if (i+1) % self.log_step == 0:
          elapsed = time.time() - start_time
          total_time = ((self.num_epochs*iters_per_epoch)-(e*iters_per_epoch+i)) * elapsed/(e*iters_per_epoch+i+1)
          epoch_time = (iters_per_epoch-i)* elapsed/(e*iters_per_epoch+i+1)
          
          epoch_time = str(datetime.timedelta(seconds=epoch_time))
          total_time = str(datetime.timedelta(seconds=total_time))
          elapsed = str(datetime.timedelta(seconds=elapsed))

          lr_tmp = []
          for param_group in self.optimizer.param_groups:
              lr_tmp.append(param_group['lr'])
          tmplr = np.squeeze(np.array(lr_tmp))

          log = "Elapsed {}/{} -- {} , Epoch [{}/{}], Iter [{}/{}], lr {}".format(
              elapsed,epoch_time,total_time, e+1, self.num_epochs, i+1, iters_per_epoch, tmplr)

          for tag, value in loss.items():
              log += ", {}: {:.4f}".format(tag, value)

        # early_stop.step(total_loss.item())
        # if early_stop.is_stop() : break
        accuracy, precision, recall, f_score = self.validation()
        print("----", f_score)

  def dagmm_step(self, input_data):
    self.dagmm.train()
    enc, dec, z, gamma = self.dagmm(input_data)
    total_loss, sample_energy, recon_error, cov_diag = self.dagmm.loss_function(input_data, dec, z, gamma, self.lambda_energy, self.lambda_cov_diag)
    
    self.reset_grad()
    total_loss.backward()

    torch.nn.utils.clip_grad_norm_(self.dagmm.parameters(), 5)
    self.optimizer.step()

    return total_loss, sample_energy, recon_error, cov_diag

  def validation(self):
    self.dagmm.eval()
    ########################################################################
    ## use train data
    N = 0; mu_sum = 0; cov_sum = 0; gamma_sum = 0

    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)
      
      batch_gamma_sum = torch.sum(gamma, dim=0)
      
      gamma_sum += batch_gamma_sum
      mu_sum += mu * batch_gamma_sum.unsqueeze(-1) # keep sums of the numerator only
      cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1) # keep sums of the numerator only
      
      N += input_data.size(0)
        
    train_phi = gamma_sum / N
    train_mu = mu_sum / gamma_sum.unsqueeze(-1)
    train_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)
    
    # print("N:",N)
    # print("phi :\n",train_phi)
    # print("mu :\n",train_mu)
    # print("cov :\n",train_cov)

    train_energy = []
    train_labels = []
    train_z = []
    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov, size_average=False)
      
      train_energy.append(sample_energy.data.cpu().numpy())
      train_z.append(z.data.cpu().numpy())
      # train_labels.append(labels.numpy())

    train_energy = np.concatenate(train_energy,axis=0)
    train_z = np.concatenate(train_z,axis=0)
    # train_labels = np.concatenate(train_labels,axis=0)

    ########################################################################
    ## validation data
    val_energy = []; val_labels = []; val_z = []
    for it, (input_data, labels) in enumerate(self.data_loader_v):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov, size_average=False)
      val_energy.append(sample_energy.data.cpu().numpy())
      val_z.append(z.data.cpu().numpy())
      val_labels.append(labels.numpy())

    val_energy = np.concatenate(val_energy,axis=0)
    val_z = np.concatenate(val_z,axis=0)
    val_labels = np.concatenate(val_labels,axis=0)

    combined_energy = np.concatenate([train_energy, val_energy], axis=0)
    print("<<<<<<<<<", len(combined_energy))
    # combined_labels = np.concatenate([train_labels, val_labels, test_labels], axis=0)
    print(np.percentile(train_energy, 90), np.percentile(val_energy, 90))
    thresh = np.percentile(combined_energy * 10000, 100 - 0.01) / 10000
    print("Threshold :", thresh)

    pred = (val_energy > thresh).astype(int)
    gt = val_labels.astype(int)

    from sklearn.metrics import precision_recall_fscore_support as prf, accuracy_score
    accuracy = accuracy_score(gt,pred)

    precision, recall, f_score, support = prf(gt, pred, average='macro')
    print("Accuracy : {:0.4f}, Precision : {:0.4f}, Recall : {:0.4f}, F-score : {:0.4f}".format(accuracy, precision, recall, f_score))
    return accuracy, precision, recall, f_score

  def test(self):
    print("======================TEST MODE======================")
    self.dagmm.eval()

    ########################################################################
    ## train
    N = 0; mu_sum = 0; cov_sum = 0; gamma_sum = 0
    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)
      
      batch_gamma_sum = torch.sum(gamma, dim=0)
      
      gamma_sum += batch_gamma_sum
      mu_sum += mu * batch_gamma_sum.unsqueeze(-1) # keep sums of the numerator only
      cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1) # keep sums of the numerator only
      
      N += input_data.size(0)
        
    train_phi = gamma_sum / N
    train_mu = mu_sum / gamma_sum.unsqueeze(-1)
    train_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)
    
    # print("N:",N)
    # print("phi :\n",train_phi)
    # print("mu :\n",train_mu)
    # print("cov :\n",train_cov)

    train_energy = []
    train_labels = []
    train_z = []
    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov, size_average=False)
      
      train_energy.append(sample_energy.data.cpu().numpy())
      train_z.append(z.data.cpu().numpy())
      # train_labels.append(labels.numpy())

    train_energy = np.concatenate(train_energy,axis=0)
    train_z = np.concatenate(train_z,axis=0)
    # train_labels = np.concatenate(train_labels,axis=0)
    ########################################################################
    ## validation
    N = 0; mu_sum = 0; cov_sum = 0; gamma_sum = 0

    for it, (input_data, labels) in enumerate(self.data_loader_v):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)
      
      batch_gamma_sum = torch.sum(gamma, dim=0)
      
      gamma_sum += batch_gamma_sum
      mu_sum += mu * batch_gamma_sum.unsqueeze(-1) # keep sums of the numerator only
      cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1) # keep sums of the numerator only
      
      N += input_data.size(0)
        
    val_phi = gamma_sum / N
    val_mu = mu_sum / gamma_sum.unsqueeze(-1)
    val_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)
    
    # print("N:",N)
    # print("phi :\n",train_phi)
    # print("mu :\n",train_mu)
    # print("cov :\n",train_cov)

    val_energy = []
    val_labels = []
    val_z = []
    for it, (input_data, labels) in enumerate(self.data_loader):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, phi=val_phi, mu=val_mu, cov=val_cov, size_average=False)
      
      val_energy.append(sample_energy.data.cpu().numpy())
      val_z.append(z.data.cpu().numpy())
      val_labels.append(labels.numpy())

    val_energy = np.concatenate(val_energy,axis=0)
    val_z = np.concatenate(val_z,axis=0)
    val_labels = np.concatenate(val_labels,axis=0)

    ########################################################################
    ## test
    test_energy = []; test_labels = []; test_z = []
    for it, (input_data, labels) in enumerate(self.data_loader_test):
      input_data = self.to_var(input_data)
      enc, dec, z, gamma = self.dagmm(input_data)
      sample_energy, cov_diag = self.dagmm.compute_energy(z, size_average=False)
      test_energy.append(sample_energy.data.cpu().numpy())
      test_z.append(z.data.cpu().numpy())
      # test_labels.append(labels.numpy())

    test_energy = np.concatenate(test_energy,axis=0)
    test_z = np.concatenate(test_z,axis=0)
    # test_labels = np.concatenate(test_labels,axis=0)

    combined_energy = np.concatenate([train_energy, val_energy, test_energy], axis=0)
    # combined_labels = np.concatenate([train_labels, val_labels, test_labels], axis=0)

    thresh = np.percentile(combined_energy, 100 - 2)
    # print("Threshold :", thresh)

    pred = (test_energy > thresh).astype(int)
    # gt = test_labels.astype(int)
    print("unique:", set(pred))
    from sklearn.metrics import precision_recall_fscore_support as prf, accuracy_score
    # accuracy = accuracy_score(gt,pred)
    # precision, recall, f_score, support = prf(gt, pred, average='binary')
    # print("Accuracy : {:0.4f}, Precision : {:0.4f}, Recall : {:0.4f}, F-score : {:0.4f}".format(accuracy, precision, recall, f_score))
    # return accuracy, precision, recall, f_score
    return pred, val_z, val_labels

In [ ]:
class Cholesky(torch.autograd.Function):
    def forward(ctx, a):
      # l = torch.potrf(a, False)
      try:
        # l = torch.linalg.cholesky(a, False)
        l = torch.cholesky(a, False)
        ctx.save_for_backward(l)
      except:
        print("-------")
        print(a)
        print("-------")
      return l
    def backward(ctx, grad_output):
      l, = ctx.saved_variables
      linv = l.inverse()
      inner = torch.tril(torch.mm(l.t(), grad_output)) * torch.tril(
          1.0 - Variable(l.data.new(l.size(1)).fill_(0.51).diag()))
      s = torch.mm(linv.t(), torch.mm(inner, linv))
      return s
    
class DaGMM(nn.Module):
    """Residual Block."""
    def __init__(self, device, n_gmm = 2, latent_dim=3):
      super(DaGMM, self).__init__()
      self.device = device
      
      # layers = []
      # layers += [nn.Linear(30, 64)]
      # layers += [nn.BatchNorm1d(64)]
      # layers += [nn.LeakyReLU()]
      # layers += [nn.Linear(64,128)]
      # layers += [nn.BatchNorm1d(128)]
      # layers += [nn.LeakyReLU()]
      # self.encoder = nn.Sequential(*layers)
    
      # layers = []
      # layers += [nn.Linear(128, 64)]
      # layers += [nn.BatchNorm1d(64)]
      # layers += [nn.LeakyReLU()]
      # layers += [nn.Linear(64,30)]
      # self.decoder = nn.Sequential(*layers)
      
      layers = []
      layers += [nn.Linear(30,60)] #####
      layers += [nn.Tanh()]        
      layers += [nn.Linear(60,30)]
      layers += [nn.Tanh()]        
      layers += [nn.Linear(30,10)]
      layers += [nn.Tanh()]        
      layers += [nn.Linear(10,1)]
      self.encoder = nn.Sequential(*layers)

      layers = []
      layers += [nn.Linear(1,10)]
      layers += [nn.Tanh()]        
      layers += [nn.Linear(10,30)]
      layers += [nn.Tanh()]        
      layers += [nn.Linear(30,60)]
      layers += [nn.Tanh()]        
      layers += [nn.Linear(60,30)]
      self.decoder = nn.Sequential(*layers)

      layers = []
      layers += [nn.Linear(latent_dim,10)]
      layers += [nn.Tanh()]        
      layers += [nn.Dropout(p=0.5)]        
      layers += [nn.Linear(10,n_gmm)]
      layers += [nn.Softmax(dim=1)]
      self.estimation = nn.Sequential(*layers)

      self.register_buffer("phi", torch.zeros(n_gmm))
      self.register_buffer("mu", torch.zeros(n_gmm,latent_dim))
      self.register_buffer("cov", torch.zeros(n_gmm,latent_dim,latent_dim))

    def relative_euclidean_distance(self, a, b):
      return (a-b).norm(2, dim=1) / a.norm(2, dim=1)

    def forward(self, x):
      enc = self.encoder(x)
      dec = self.decoder(enc)

      rec_cosine = F.cosine_similarity(x, dec, dim=1)
      rec_euclidean = self.relative_euclidean_distance(x, dec)
      z = torch.cat([enc, rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)

      print("======================= z ==================\n", z)
      print("======================= enc ==================\n", enc)
      print("======================= dec ==================\n", dec)
      print("======================= cosines ==================\n", rec_cosine)
      print("======================= euclidean ==================\n", rec_euclidean)
      print("xxxx\n", x, "\n")

      gamma = self.estimation(z)
      return enc, dec, z, gamma

    def compute_gmm_params(self, z, gamma):
      N = gamma.size(0)
      # K
      sum_gamma = torch.sum(gamma, dim=0)

      # K
      phi = (sum_gamma / N)
      self.phi = phi.data

      # K x D
      mu = torch.sum(gamma.unsqueeze(-1) * z.unsqueeze(1), dim=0) / sum_gamma.unsqueeze(-1)
      self.mu = mu.data
      # z = N x D
      # mu = K x D
      # gamma N x K

      # z_mu = N x K x D
      z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

      # z_mu_outer = N x K x D x D
      z_mu_outer = z_mu.unsqueeze(-1) * z_mu.unsqueeze(-2)

      # K x D x D
      cov = torch.sum(gamma.unsqueeze(-1).unsqueeze(-1) * z_mu_outer, dim = 0) / sum_gamma.unsqueeze(-1).unsqueeze(-1)
      self.cov = cov.data
      return phi, mu, cov
        
    def compute_energy(self, z, phi=None, mu=None, cov=None, size_average=True):
      if phi is None : phi = self.phi #self.to_var(self.phi)
      if mu is None : mu = self.mu # self.to_var(self.mu)
      if cov is None : cov = self.cov #self.to_var(self.cov)

      k, D, _ = cov.size()
      z_mu = (z.unsqueeze(1)- mu.unsqueeze(0))

      cov_inverse = []
      det_cov = []
      cov_diag = 0
      eps = 1e-12
      for i in range(k):
        # K x D x D
        # cov_k = cov[i] + self.to_var(torch.eye(D)*eps)
        cov_k = cov[i] + (torch.eye(D)*eps).to(device = self.device)
        cov_inverse.append(torch.inverse(cov_k).unsqueeze(0))
        
        if np.isnan((cov_k.cpu() * (2*np.pi))[0,0].item()) : 
          print(cov_k)
          import time; time.sleep(30)
          continue
        #det_cov.append(np.linalg.det(cov_k.data.cpu().numpy()* (2*np.pi)))
        det_cov.append((Cholesky.apply(cov_k.cpu() * (2*np.pi)).diag().prod()).unsqueeze(0))
        cov_diag = cov_diag + torch.sum(1 / cov_k.diag())

      # K x D x D
      cov_inverse = torch.cat(cov_inverse, dim=0)
      # K
      det_cov = torch.cat(det_cov).cuda()
      #det_cov = to_var(torch.from_numpy(np.float32(np.array(det_cov))))

      # N x K
      exp_term_tmp = -0.5 * torch.sum(torch.sum(z_mu.unsqueeze(-1) * cov_inverse.unsqueeze(0), dim=-2) * z_mu, dim=-1)
      # for stability (logsumexp)
      max_val = torch.max((exp_term_tmp).clamp(min=0), dim=1, keepdim=True)[0]
      exp_term = torch.exp(exp_term_tmp - max_val)

      # sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (det_cov).unsqueeze(0), dim = 1) + eps)
      sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt(det_cov)).unsqueeze(0), dim = 1) + eps)
      # sample_energy = -max_val.squeeze() - torch.log(torch.sum(phi.unsqueeze(0) * exp_term / (torch.sqrt((2*np.pi)**D * det_cov)).unsqueeze(0), dim = 1) + eps)

      if size_average : sample_energy = torch.mean(sample_energy)
      return sample_energy, cov_diag

    def to_var(x, volatile=False):
        if torch.cuda.is_available() : x = x.cuda()
        print("=====", x)
        import time
        # time.sleep(30)
        return Variable(x, volatile=volatile)

    def loss_function(self, x, x_hat, z, gamma, lambda_energy, lambda_cov_diag):
        recon_error = torch.mean((x - x_hat) ** 2)
        phi, mu, cov = self.compute_gmm_params(z, gamma)
        sample_energy, cov_diag = self.compute_energy(z, phi, mu, cov)
        loss = recon_error + lambda_energy * sample_energy + lambda_cov_diag * cov_diag
        return loss, sample_energy, recon_error, cov_diag


In [ ]:
import os
import argparse
# from solver import Solver
# from data_loader import get_loader
from torch.backends import cudnn
# from utils import *

def str2bool(v):
    return v.lower() in ('true')

def main(config):
    # For fast training
    cudnn.benchmark = True

    # Create directories if not exist
    # mkdir(config.log_path)
    # mkdir(config.model_save_path)
    # wandb.init()
    data_loader, data_loader_v, data_loader_test = get_loader(config.data_path, batch_size=config.batch_size, mode=config.mode)
    
    import torch
    USE_CUDA = torch.cuda.is_available()
    device = torch.device('cuda:0' if USE_CUDA else 'cpu')

    # Solver
    solver = Solver(device, data_loader, data_loader_v, data_loader_test, vars(config))

    if config.mode == 'train' : 
      solver.train()
      return solver
    elif config.mode == 'test' : 
      pred, val_z, val_labels = solver.test()
      return solver, pred, val_z, val_labels

if __name__ == '__main__':
    import easydict
    args = easydict.EasyDict({
        "lr" : 1e-4,
        "num_epochs" : 100,
        "batch_size" : 512,
        "gmm_k" : 4,
        "lambda_energy" : 0.1,
        "lambda_cov_diag" : 0.005,
        "pretrained_model" : '',
        # "pretrained_model" : None,
        "mode" : 'train',
        # "mode" : "test",
        "use_tensorboard" : False,
        "data_path" : "./",
        "log_path" : './logs',
        "model_save_path" : './models',
        "log_step" : 100,
        "sample_step" : 194,
        "model_save_step" : 194
    })
    config = args
    # args = vars(config)
    print('------------ Options -------------')
    for k, v in sorted(args.items()):
        print('%s: %s' % (str(k), str(v)))
    print('-------------- End ----------------')

    solver, pred, val_z, val_labels = main(config)

------------ Options -------------
batch_size: 512
data_path: ./
gmm_k: 4
lambda_cov_diag: 0.005
lambda_energy: 0.1
log_path: ./logs
log_step: 100
lr: 0.0001
mode: train
model_save_path: ./models
model_save_step: 194
num_epochs: 100
pretrained_model: 
sample_step: 194
use_tensorboard: False
-------------- End ----------------
DaGMM
The number of parameters: 8135


  0%|          | 0/223 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  from ipykernel import kernelapp as app
1 epoch's total_loss : 30.178747177124023:   0%|          | 0/223 [00:00<?, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [ 0.2079]], device='cuda:0', grad_fn=<AddmmBackward0>)
======================= dec ==================
 tensor([[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [-0.0337,  0.0352,  0.1650,  ...,  0.0065, -0.0559,  0.1204],
        ...,
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [-0.0499,  0.0515,  0.1521,  ...,  0.0130, -0.0423,  0.1379],
        [-0.0392,  0.0405,  0.1606,  ...,  0.0085, -0.0515,  0.1263]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
======================= cosines ==================
 tensor([        nan,         nan, -5.4601e-03,  1.1438e-01, -1.5096e-01,
         8.0295e-02,         nan,         nan,         nan, -6.0566e-02,
         2.6793e-02,  9.3122e-03,  1.0311e-01,  1.5064e-01,         nan,
                nan,  1.0775e-01,         nan,  2.3041e-01,         nan,
                nan,     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1 epoch's total_loss : 30.178747177124023:   0%|          | 1/223 [00:33<2:04:38, 33.69s/it]

======================= z ==================
 tensor([[ 0.1705,  1.0568, -0.3841],
        [    nan,     nan,     nan],
        [ 0.2423,  1.0101,  0.0661],
        ...,
        [ 0.2914,  0.9841,  0.1992],
        [ 0.3388,  1.0211, -0.0859],
        [ 0.2567,  0.9910,  0.1342]], device='cuda:0', grad_fn=<CatBackward0>)
======================= enc ==================
 tensor([[ 0.1705],
        [    nan],
        [ 0.2423],
        [    nan],
        [    nan],
        [ 0.2753],
        [ 0.2510],
        [ 0.2392],
        [    nan],
        [    nan],
        [ 0.2179],
        [ 0.2498],
        [ 0.3642],
        [ 0.1350],
        [    nan],
        [ 0.3265],
        [    nan],
        [ 0.2092],
        [    nan],
        [ 0.2600],
        [    nan],
        [ 0.2238],
        [    nan],
        [    nan],
        [    nan],
        [ 0.2450],
        [    nan],
        [ 0.2498],
        [ 0.3317],
        [    nan],
        [    nan],
        [ 0.4329],
        [    nan],
  

...

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [    nan],
        [    nan],
        [ 0.2646],
        [    nan],
        [ 0.0591],
        [    nan],
        [    nan],
        [ 0.2205],
        [    nan],
        [    nan],
        [ 0.3828],
        [    nan],
        [    nan],
        [    nan],
        [    nan],
        [    nan],
        [    nan],
        [ 0.3480],
        [ 0.1107],
        [ 0.2389],
        [ 0.2856],
        [    nan],
        [ 0.4205],
        [    nan],
        [    nan],
        [    nan],
        [    nan],
        [    nan],
        [ 0.1414],
        [ 0.0850],
        [    nan],
        [    nan],
        [    nan],
        [ 0.2596],
        [    nan],
        [    nan],
        [ 0.2652],
        [    nan],
        [ 0.2707],
        [    nan],
        [ 0.2158],
        [ 0.3053],
        [ 0.2627],
        [    nan],
        [    nan],
        [ 0.2124],
        [ 0.1365],
        [    nan],
        [    nan],
        [ 0.2101],
        [ 0.22

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1 epoch's total_loss : 25.455257415771484:   2%|▏         | 5/223 [02:49<2:03:30, 33.99s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  from ipykernel import kernelapp as app
1 epoch's total_loss : 24.605087280273438:   2%|▏         | 5/223 [02:49<2:03:30, 33.99s/it]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [0.2282],
        [   nan],
        [0.2702],
        [   nan],
        [   nan],
        [   nan],
        [   nan],
        [   nan],
        [0.2465],
        [0.2390],
        [0.3081],
        [   nan],
        [   nan],
        [   nan],
        [   nan],
        [   nan],
        [   nan],
        [0.2270],
        [0.2212],
        [   nan],
        [   nan],
        [   nan],
        [   nan],
        [0.2797],
        [   nan],
        [   nan],
        [0.2093],
        [   nan],
        [0.2571],
        [0.2767],
        [0.0550],
        [0.1118],
        [0.1427],
        [   nan],
        [   nan],
        [0.2156],
        [   nan],
        [0.2503],
        [   nan],
        [0.0910],
        [   nan],
        [   nan],
        [0.2173],
        [0.2400],
        [0.2350],
        [   nan],
        [0.3228],
        [   nan],
        [0.2469],
        [0.1707],
        [0.2732],
        [   nan],
        [   nan],
        [ 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1 epoch's total_loss : 24.605087280273438:   3%|▎         | 6/223 [03:23<2:03:32, 34.16s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  from ipykernel import kernelapp as app
1 epoch's total_loss : 24.84026336669922:   3%|▎         | 6/223 [03:24<2:03:32, 34.16s/it] 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [    nan],
        [ 0.1965],
        [    nan],
        [    nan],
        [ 0.3037],
        [ 0.3289],
        [    nan],
        [ 0.1352],
        [ 0.2388],
        [ 0.0390],
        [    nan],
        [    nan],
        [ 0.2373],
        [    nan],
        [ 0.2589],
        [ 0.1533],
        [ 0.2295],
        [ 0.2266],
        [    nan],
        [ 0.2084],
        [ 0.3593],
        [ 0.2216],
        [    nan],
        [ 0.2260],
        [ 0.2698],
        [ 0.2066],
        [ 0.1934],
        [    nan],
        [    nan],
        [    nan],
        [    nan],
        [ 0.2381],
        [    nan],
        [    nan],
        [ 0.2392],
        [    nan],
        [    nan],
        [ 0.2097],
        [    nan],
        [    nan],
        [    nan],
        [ 0.1680],
        [ 0.3300],
        [ 0.2631],
        [ 0.1437],
        [ 0.1576],
        [ 0.2975],
        [ 0.1259],
        [ 0.1651],
        [ 0.1533],
        [ 0.20

In [ ]:
pred

NameError: ignored

In [ ]:
len(pred)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['Class'] = pred
submit.to_csv('./submit_0.01.csv', index=False)

NameError: ignored

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# ax.scatter(test_z[:,1],test_z[:,0], test_z[:,2], c=test_labels.astype(int))
ax.scatter(val_z[:,1],val_z[:,0], val_z[:,2], c=val_labels.astype(int), alpha=0.1)

ax.set_xlabel('Encoded')
ax.set_ylabel('Euclidean')
ax.set_zlabel('Cosine')
plt.show()
plt.savefig('fig2.png')

<IPython.core.display.Javascript object>

NameError: ignored